In [1]:
import math
from PIL import Image
from matplotlib import pyplot
import numpy as np
#функция отрисовки линии В-ЮВ
def draw_line_byX(img, x0, y0, x1, y1, color):
    dx = x1-x0
    dy = y1-y0
    d = dy/dx
    y = y0
    e = 0
    for x in range(x0, x1+1):
        img[int(x),int(y),:] = color_line
        e += math.fabs(d)
        if e >= 0.5:
            y += int(math.copysign(1,dy))
            e -= 1
    return
#функция отрисовки линии Ю-ЮВ
def draw_line_byY(img, x0, y0, x1, y1, color):
    dx = x1-x0
    dy = y1-y0
    d = dx/dy
    x = x0
    e = 0
    for y in range(y0, y1+1):
        img[int(x),int(y),:] = color_line
        e += math.fabs(d)
        if e >= 0.5:
            x += int(math.copysign(1,dx))
            e -= 1
    return
#функция отрисовки линии в общем случае
def draw_line(img, x0, y0, x1, y1, color):
    if math.fabs(x1 - x0) > math.fabs(y1 - y0):
        if x1 > x0:
            draw_line_byX(img, x0,y0,x1,y1, color)
        else:
            draw_line_byX(img, x1,y1,x0,y0, color)
    else:
        if y1 > y0:
            draw_line_byY(img, x0,y0,x1,y1, color)
        else:
            draw_line_byY(img, x1,y1,x0,y0, color)
    return
#функция отрисовки ломаной
def draw_polyline(img, x, y, color = (0,0,0)):
    n = len(x)
    for i in range(n-1):
        draw_line(img,x[i],y[i],x[i+1],y[i+1],color)
    return

In [2]:
#координаты для построения
coords = [[(1, 1), (1, 20), (10,14), (23,20), (23, 1), (1, 1)], [(5, 5), (5, 10), (12, 10), (5, 5)], [(15, 10), (20, 10), (20, 15), (15, 10)]]

#функция для заливки полигона
def fill_polygon2(img, polygon, color):
    
    #поиск наименьшей и наибольшей координаты x
    width = img.size[0]-1 #обращение к ширине изображения в строке из двух значений (ширина и высота)
    y_min = width
    y_max = 0
 
    for ring in polygon:
        for i in range(len(ring)-1):
            if ring[i][1] > y_max:
                y_max = ring[i][1]
            if ring[i][1] < y_min:
                y_min = ring[i][1]
        
    #выполнение заливки полигона
    for yk in range(y_min,  y_max):
        X = []
        for ring in polygon:
            for j in range(len(ring)-1):
                x_j  = ring[j][0]
                x_j1 = ring[j+1][0]
                y_j  = ring[j][1]
                y_j1 = ring[j+1][1]

                if (y_j < yk) != (y_j1 < yk):
                    xj = x_j + (x_j1 - x_j) * ((yk - y_j) / (y_j1-y_j))
                    X.append(xj)
               
            X.sort()

            kl = 0
        
        while kl < len(X):
            for xk in range(math.ceil(X[kl]), math.floor(X[kl+1])+1):
                px[xk, yk, :] = color
            kl += 2    

    return
#функция отрисовки границы
def stroke_polygon(img, polygon, color):
    for ring in polygon:
        for i in range(len(ring)-1):
            draw_line(img, ring[i][0], ring[i][1], ring[i+1][0], ring[i+1][1], color)
    return

In [3]:
def resample_box(img_b, img):
    ni = img_b.shape[0] #размеры буферного растра
    nj = img_b.shape[1] 
    ki = img.shape[0] #размеры выводного растра
    kj = img.shape[1]
    
    k = int(ni) / int(ki) #коэффициент сглаживания
    
    for i in range(ki):
        for j in range(kj):
            di = k * i
            dj = k * j
            s = np.array([0, 0, 0])
            for xi in range (di, di + k):
                for xj in range (dj, dj + k):
                    s += img_b[xi, xj, :]
            img[i, j, :] = s / (k**2)
    return

def draw_polygon(img_b, polygon, fill = (103, 234, 14), stroke = (1, 235, 14), k = 2):
    ki = img_b.shape[0] #размеры выводного растра
    kj = img_b.shape[1]
    img_b = np.full((ki * k, kj * k, 3), 255)
    fill_polygon2(img_b, polygon * k, fill)
    stroke_polygon2(img_b, polygon * k, stroke)
    resample_box(img_b, img)
    return

#тестирование блочного суперсемплинга
polygon = 8 * np.array([[[1, 1], [1, 20], [10,14], [23,20], [23, 1], [1, 1]], [[5, 5], [5, 10], [12, 10], [5, 5]], [[15, 10], [20, 10], [20, 15], [15, 10]]])

img = np.full((400, 400, 3), 255)

draw_polygon(img, polygon)

plt.figure(figsize = (10, 10))

plt.imshow(img)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


TypeError: 'int' object is not subscriptable